# Creating and bundling custom profilers

In [1]:
from typing import Union, Tuple, Optional
import itertools as it

import numpy as np
import pandas as pd
import statsmodels.tsa.api as sm

In [2]:
# patch it
_read_csv = pd.read_csv
pd.read_csv = lambda fpath, *args, **kwargs: _read_csv(fpath.split('/')[-1], *args, **kwargs)

## Profiling Functions

Creating a customer profiler is very straightforward. For a function to qualify as a profiler it needs:

- To be type annotated
- To have one and only one positional argument, the timeseries, which needs to be a pandas DataFrame or Series (or a union of both)
- To be decorated with `@ProfilingFunction`

Let's use a few dummy examples to better illustrate this.

In [3]:
from tslumen.profile import ProfilingFunction

In [4]:
@ProfilingFunction
def shape(data: Union[pd.Series, pd.DataFrame]) -> Tuple[int, ...]:
    return data.shape

shape

As shown by the checkmarks, the `shape` function we just created is applied to the DataFrame as a whole, as well as each individual series. 

Under the hood, the `@ProfilingFunction` decorator inspects the function and wraps it inside a class which, among other purposes, identifies the scope of action of the profiler (i.e. single Series, DataFrame or both), particularly useful when orchestrating the execution of multiple profilers. More on that later.

In [5]:
@ProfilingFunction
def acovf(data: pd.Series, adjusted: bool = False, demean: bool = True,
          fft: bool = True, missing: str = 'none', nlag: Optional[int] = None) -> pd.Series:
    est = sm.acovf(data.values, adjusted=adjusted, demean=demean, fft=fft, missing=missing, nlag=nlag)
    return pd.Series(est, index=data.index)

acovf

In [6]:
@ProfilingFunction
def euc_sim(data: pd.DataFrame, measure: str = 'rmse') -> pd.DataFrame:
    fn = lambda d: d.mean() if measure == 'mean' else np.sqrt((d**2).mean())
    res = {(a, b): fn(data[b]-data[a])
           for a, b in it.product(data.columns, data.columns)}
    return pd.Series(res).unstack(0)

euc_sim

Now the `acovf` function is for Series only, whereas `euc_sim` is for DataFrames only.

When wrapping functionality from another library, it is a good idea to expose (some of) its parameters as to make the profiler more configurable. Naturally, in some cases, it's actually a good idea to hide them from the end-user, particularly when the wrapper itself will have the logic to select the parameters.

Note that any arguments the profiling function exposes (other than the first one) need to have defaults.


Let's load a dataset and try out these profilers

In [7]:
df = pd.read_csv('https://datahub.io/core/s-and-p-500/r/data.csv', parse_dates=[0], index_col=0)
df = df[(df.index >= '1990-01-01') & (df.index < '2015-01-01')]
df.head()

,SP500,Dividend,Earnings,Consumer Price Index,Long Interest Rate,Real Price,Real Dividend,Real Earnings,PE10
Date,,,,,,,,,
1990-01-01,339.97,11.14,22.49,127.4,8.21,666.69,21.85,44.10,17.05
1990-02-01,330.45,11.23,22.08,128.0,8.47,644.99,21.92,43.10,16.51
1990-03-01,338.46,11.32,21.67,128.7,8.59,657.03,21.97,42.07,16.83
1990-04-01,338.18,11.44,21.53,128.9,8.79,655.46,22.17,41.74,16.81
1990-05-01,350.25,11.55,21.40,129.2,8.76,677.28,22.34,41.38,17.39


In [8]:
shape(df)

ProfileResult(name='shape', scope='frame', target='', start=datetime.datetime(2022, 11, 9, 14, 12, 59, 964155), end=datetime.datetime(2022, 11, 9, 14, 12, 59, 964236), success=True, warnings=[], exception=None, result=(300, 9))

In [9]:
shape(df.SP500)

ProfileResult(name='shape', scope='series', target='SP500', start=datetime.datetime(2022, 11, 9, 14, 12, 59, 998106), end=datetime.datetime(2022, 11, 9, 14, 12, 59, 998165), success=True, warnings=[], exception=None, result=(300,))

In [10]:
acovf(df.SP500)

Date,1990-01-01,1990-02-01,1990-03-01,1990-04-01,1990-05-01,1990-06-01,1990-07-01,1990-08-01,1990-09-01,1990-10-01,...,2014-03-01,2014-04-01,2014-05-01,2014-06-01,2014-07-01,2014-08-01,2014-09-01,2014-10-01,2014-11-01,2014-12-01
0,179217.457496,175888.867311,172240.056519,168900.130792,165215.340345,161487.048775,157530.850357,153705.459385,149985.479096,146125.05111,...,-21330.750303,-19308.50963,-17303.253341,-15288.805447,-13242.503765,-11143.461295,-9034.245959,-6815.498383,-4723.390484,-2356.619013


In [11]:
euc_sim(df, 'rmse')

,Consumer Price Index,Dividend,Earnings,Long Interest Rate,PE10,Real Dividend,Real Earnings,Real Price,SP500
Consumer Price Index,0.000000,165.486344,135.796101,181.364722,161.318178,159.546683,122.081817,1247.349170,935.855699
Dividend,165.486344,0.000000,35.661179,17.019169,11.701936,6.940290,48.498664,1407.033108,1093.831545
Earnings,135.796101,35.661179,0.000000,51.538956,34.956319,31.895157,15.897771,1375.411112,1060.978612
Long Interest Rate,181.364722,17.019169,51.538956,0.000000,21.497434,22.065529,64.027759,1422.551992,1110.110884
PE10,161.318178,11.701936,34.956319,21.497434,0.000000,8.751860,45.141560,1401.371581,1089.657346
Real Dividend,159.546683,6.940290,31.895157,22.065529,8.751860,0.000000,43.418904,1401.303869,1088.640134
Real Earnings,122.081817,48.498664,15.897771,64.027759,45.141560,43.418904,0.000000,1361.394243,1048.086806
Real Price,1247.349170,1407.033108,1375.411112,1422.551992,1401.371581,1401.303869,1361.394243,0.000000,364.090297
SP500,935.855699,1093.831545,1060.978612,1110.110884,1089.657346,1088.640134,1048.086806,364.090297,0.000000


All decorated functions return a `ProfileResult` object, which is a mere wrapper around the function's result to capture additional details like the name of the function, execution times, exceptions, etc. 

It can be used as a class or cast as a dictionary.

In [12]:
pr = shape(df.SP500)
dict(pr)

{'name': 'shape',
 'scope': 'series',
 'target': 'SP500',
 'start': datetime.datetime(2022, 11, 9, 14, 13, 0, 160374),
 'end': datetime.datetime(2022, 11, 9, 14, 13, 0, 160422),
 'success': True,
 'warnings': [],
 'exception': None,
 'result': (300,)}

In [13]:
pr.start

datetime.datetime(2022, 11, 9, 14, 13, 0, 160374)

## Bundling profilers together

Single profilers (i.e. `ProfilingFunction` decorated functions) can be bundled together in one single class, which will deal with orchestrating their execution and collecting the results.

A `DefaultProfiler` is included in tslumen, essentially bundling all the profiling functions included in the package.

Creating a custom bundle is simply a matter of creating a subclass of `BundledProfiler` and defining the class variable `_profilers` as a list of the profiling functions that are to be included. Note that these need to have unique names, as the name is used to store the output of the function.

Let's start by having a look at the `DefaultProfiler`.

In [14]:
from tslumen.profile import DefaultProfiler

In [15]:
DefaultProfiler.get_profilers()

{'df_scaled': <tslumen.profile.base.ProfilingFunction at 0x7f95e875e7c0>,
 'dt_end': <tslumen.profile.base.ProfilingFunction at 0x7f95e875edc0>,
 'dt_start': <tslumen.profile.base.ProfilingFunction at 0x7f95e875e700>,
 'freq': <tslumen.profile.base.ProfilingFunction at 0x7f95e8760b80>,
 'infinite': <tslumen.profile.base.ProfilingFunction at 0x7f95e8760f40>,
 'length': <tslumen.profile.base.ProfilingFunction at 0x7f95e875ea30>,
 'missing': <tslumen.profile.base.ProfilingFunction at 0x7f95e8760a30>,
 'n_series': <tslumen.profile.base.ProfilingFunction at 0x7f95e875ebb0>,
 'period': <tslumen.profile.base.ProfilingFunction at 0x7f95e875ed60>,
 'sample': <tslumen.profile.base.ProfilingFunction at 0x7f95e87603d0>,
 'sz_total': <tslumen.profile.base.ProfilingFunction at 0x7f95e875e2b0>,
 'zeros': <tslumen.profile.base.ProfilingFunction at 0x7f95e875e520>,
 'cov': <tslumen.profile.base.ProfilingFunction at 0x7f95e86f0a60>,
 'iqr': <tslumen.profile.base.ProfilingFunction at 0x7f95e86f4b80>,
 'k

The class method `get_profilers` returns a dictionary with all the `ProfilingFunction` bundled in this `BundledProfiler`

As the name suggests, the class method `get_config_defaults` returns a dictionary with the default parameters of each `ProfilingFunction`. It is useful for understanding exactly which parameter defaults can be overridden.



In [16]:
DefaultProfiler.get_config_defaults()

{'df_scaled': {},
 'dt_end': {},
 'dt_start': {},
 'freq': {},
 'infinite': {},
 'length': {},
 'missing': {},
 'n_series': {},
 'period': {},
 'sample': {'sample_size': 10},
 'sz_total': {'memory_deep': True},
 'zeros': {},
 'cov': {},
 'iqr': {},
 'kurtosis': {},
 'mad': {},
 'maximum': {},
 'mean': {},
 'median': {},
 'minimum': {},
 'q25': {},
 'q50': {},
 'q75': {},
 'skew': {},
 'std': {},
 'var': {},
 'adfuller_stationarity': {'confidence_level': 0.05},
 'jarque_bera_normality': {'confidence_level': 0.05},
 'kpss_stationarity': {'confidence_level': 0.05},
 'levene_constant_variance': {'confidence_level': 0.05},
 'ljungbox_autocorrelation': {'n_lags': None, 'confidence_level': 0.05},
 'omnibus_normality': {'confidence_level': 0.05},
 'binned': {'nbins': None},
 'pd_percentiles': {},
 'pd_quantiles': {},
 'acf': {'lags': 40,
  'adjusted': False,
  'fft': False,
  'alpha': 0.05,
  'missing': 'none'},
 'acf_1d': {'lags': 40,
  'adjusted': False,
  'fft': False,
  'alpha': 0.05,
  'm

Let us now create our own profiler by bundling the functions defined above.

In [17]:
from tslumen.profile import BundledProfiler


class MyProfiler(BundledProfiler):
    _profilers = [shape, acovf, euc_sim]

In [18]:
MyProfiler.get_profilers()

{'shape': <tslumen.profile.base.ProfilingFunction at 0x7f95e5a9d8b0>,
 'acovf': <tslumen.profile.base.ProfilingFunction at 0x7f95e595aa60>,
 'euc_sim': <tslumen.profile.base.ProfilingFunction at 0x7f95e5915490>}

In [19]:
MyProfiler.get_config_defaults()

{'shape': {},
 'acovf': {'adjusted': False,
  'demean': True,
  'fft': True,
  'missing': 'none',
  'nlag': None},
 'euc_sim': {'measure': 'rmse'}}

When instantiating the bundled profiler, the configs will be set, essentially by taking the defaults of each `ProfilingFunction` and overriding any parameters explicitly provided as an argument.

In [20]:
# default configs were set
profiler1 = MyProfiler()
profiler1.config

{'shape': {},
 'acovf': {'adjusted': False,
  'demean': True,
  'fft': True,
  'missing': 'none',
  'nlag': None},
 'euc_sim': {'measure': 'rmse'}}

In [21]:
# euc_sim's measure was overriden
profiler2 = MyProfiler(config={'euc_sim': {'measure': 'mean'}})
profiler2.config

{'shape': {},
 'acovf': {'adjusted': False,
  'demean': True,
  'fft': True,
  'missing': 'none',
  'nlag': None},
 'euc_sim': {'measure': 'mean'}}

With our profiler instantiated, we can now profile our data. The bundled profiler will take care of orchestrating the execution of the profiling functions and collecting the results. Depending on the function's scope, each will be called with the dataset or once per series.

In [22]:
pr1 = profiler1.profile(df)

Profiling:   0%|          | 0/20 [00:00<?, ?it/s]

In [23]:
pr2 = profiler2.profile(df)

Profiling:   0%|          | 0/20 [00:00<?, ?it/s]

In [24]:
pr1

,Profiler,Scope,Target,Start,End,Duration,Succeeded,Exceptions,# Runs
0,shape,frame,,2022-11-09 14:13:03.273098,2022-11-09 14:13:03.273204,0 days 00:00:00.000106,True,None,1
1,euc_sim,frame,,2022-11-09 14:13:03.252686,2022-11-09 14:13:03.286046,0 days 00:00:00.033360,True,None,1
2,shape,series,SP500,2022-11-09 14:13:03.311705,2022-11-09 14:13:03.311827,0 days 00:00:00.000122,True,None,1
3,shape,series,Dividend,2022-11-09 14:13:03.252150,2022-11-09 14:13:03.252262,0 days 00:00:00.000112,True,None,1
4,shape,series,Earnings,2022-11-09 14:13:03.259253,2022-11-09 14:13:03.259377,0 days 00:00:00.000124,True,None,1
5,shape,series,Consumer Price Index,2022-11-09 14:13:03.381316,2022-11-09 14:13:03.381450,0 days 00:00:00.000134,True,None,1
6,shape,series,Long Interest Rate,2022-11-09 14:13:03.314238,2022-11-09 14:13:03.314358,0 days 00:00:00.000120,True,None,1
7,shape,series,Real Price,2022-11-09 14:13:03.343129,2022-11-09 14:13:03.343272,0 days 00:00:00.000143,True,None,1
8,shape,series,Real Dividend,2022-11-09 14:13:03.524819,2022-11-09 14:13:03.525077,0 days 00:00:00.000258,True,None,1
9,shape,series,Real Earnings,2022-11-09 14:13:03.441669,2022-11-09 14:13:03.441782,0 days 00:00:00.000113,True,None,1


Results are held in a `BundledResult` object, which includes the same details as `ProfileResult`. Under the `result` attribute you'll find:


- `exec_details` a DataFrame with all the executions
- `config` the configurations passed onto this execution
- `frame` a dictionary with the results of each profiling function whose scope is DataFrame
- `series` a dictionary with each of the series in the DataFrame, and for each series a dictionary with the results of the series profiling functions

In [25]:
pr1.result.frame['euc_sim']

,Consumer Price Index,Dividend,Earnings,Long Interest Rate,PE10,Real Dividend,Real Earnings,Real Price,SP500
Consumer Price Index,0.000000,165.486344,135.796101,181.364722,161.318178,159.546683,122.081817,1247.349170,935.855699
Dividend,165.486344,0.000000,35.661179,17.019169,11.701936,6.940290,48.498664,1407.033108,1093.831545
Earnings,135.796101,35.661179,0.000000,51.538956,34.956319,31.895157,15.897771,1375.411112,1060.978612
Long Interest Rate,181.364722,17.019169,51.538956,0.000000,21.497434,22.065529,64.027759,1422.551992,1110.110884
PE10,161.318178,11.701936,34.956319,21.497434,0.000000,8.751860,45.141560,1401.371581,1089.657346
Real Dividend,159.546683,6.940290,31.895157,22.065529,8.751860,0.000000,43.418904,1401.303869,1088.640134
Real Earnings,122.081817,48.498664,15.897771,64.027759,45.141560,43.418904,0.000000,1361.394243,1048.086806
Real Price,1247.349170,1407.033108,1375.411112,1422.551992,1401.371581,1401.303869,1361.394243,0.000000,364.090297
SP500,935.855699,1093.831545,1060.978612,1110.110884,1089.657346,1088.640134,1048.086806,364.090297,0.000000


In [26]:
pr2.result.frame['euc_sim']

,Consumer Price Index,Dividend,Earnings,Long Interest Rate,PE10,Real Dividend,Real Earnings,Real Price,SP500
Consumer Price Index,0.000000,163.441100,134.428367,178.143767,157.812267,156.987300,119.670567,-1177.380400,-847.525000
Dividend,-163.441100,0.000000,-29.012733,14.702667,-5.628833,-6.453800,-43.770533,-1340.821500,-1010.966100
Earnings,-134.428367,29.012733,0.000000,43.715400,23.383900,22.558933,-14.757800,-1311.808767,-981.953367
Long Interest Rate,-178.143767,-14.702667,-43.715400,0.000000,-20.331500,-21.156467,-58.473200,-1355.524167,-1025.668767
PE10,-157.812267,5.628833,-23.383900,20.331500,0.000000,-0.824967,-38.141700,-1335.192667,-1005.337267
Real Dividend,-156.987300,6.453800,-22.558933,21.156467,0.824967,0.000000,-37.316733,-1334.367700,-1004.512300
Real Earnings,-119.670567,43.770533,14.757800,58.473200,38.141700,37.316733,0.000000,-1297.050967,-967.195567
Real Price,1177.380400,1340.821500,1311.808767,1355.524167,1335.192667,1334.367700,1297.050967,0.000000,329.855400
SP500,847.525000,1010.966100,981.953367,1025.668767,1005.337267,1004.512300,967.195567,-329.855400,0.000000


In [27]:
pr1.result.frame['euc_sim'].equals(pr2.result.frame['euc_sim'])

False

As we can see, the parameter is being passed on correctly to `euc_sim`.

## Closing thoughts


This notebook illustrates how simple it is to create custom profilers and bundles.

It's up to the user to mix and match whichever profilers are relevant to their needs, pre-canned ones available under `tslumen.profile` or user-defined functions, provided they meet the conventions listed above and are decorated using `@ProfilingFunction`. 